**Extracting data from API**

In [8]:
import requests # library for making HTTP requests
import pandas as pd # library for data analysis
import datetime as dt # library for handling date and time objects
from datetime import datetime, timedelta
import csv

In [9]:
# Read the keys and tokens into a dictionary
my_dict = {}

with open(".env", "r") as f:
    for line in f:
        key,val = line.split('=')
        my_dict[key.strip()] = val.strip()

In [10]:
api_key = my_dict['api_key']

In [11]:
import csv
import requests
from datetime import datetime, timedelta
import time

# Set up the base parts of the request
url = "https://skyscanner80.p.rapidapi.com/api/v1/flights/search-one-way"
headers = {
    "X-RapidAPI-Key": api_key,
    "X-RapidAPI-Host": "skyscanner80.p.rapidapi.com"
}

# CSV file setup
csv_file_name = 'flight_data.csv'
with open(csv_file_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Airline', 'Flight Number', 'Source City', 'Departure Time', 'Number of Stops', 'Arrival Time', 'Destination City', 'Duration (Minutes)', 'Price'])

    # Use today's date as the starting point
    start_date = datetime.now()

    # Loop over the next 2 days for demonstration (change to 30 for actual use)
    for offset in range(300):
        # Calculate the date for the current iteration
        depart_date = start_date + timedelta(days=offset)
        # Format the date in YYYY-MM-DD format
        formatted_date = depart_date.strftime('%Y-%m-%d')

        # Update the query string with the new date
        querystring = {
            "fromId": "eyJzIjoiTEFYQSIsImUiOiIyNzUzNjIxMSIsImgiOiIyNzUzNjIxMSJ9",
            "toId": "eyJzIjoiTE9ORCIsImUiOiIyNzU0NDAwOCIsImgiOiIyNzU0NDAwOCJ9",
            "departDate": formatted_date,
            "adults": "1",
            "currency": "USD",
            "market": "US",
            "locale": "en-US"
        }

        # Make the request
        response = requests.get(url, headers=headers, params=querystring)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            itineraries = data['data']['itineraries'] if 'data' in data and 'itineraries' in data['data'] else []

            # Process each itinerary
            for itinerary in itineraries:
                price = itinerary['price']['formatted']
                for leg in itinerary['legs']:
                    airline = leg['carriers']['marketing'][0]['name']
                    flight_number = leg['segments'][0]['flightNumber']
                    source_city = leg['origin']['city']
                    departure_time = leg['departure']
                    number_of_stops = leg['stopCount']
                    arrival_time = leg['arrival']
                    destination_city = leg['destination']['city']
                    duration = leg['durationInMinutes']

                    # Write the details to the CSV
                    writer.writerow([airline, flight_number, source_city, departure_time, number_of_stops, arrival_time, destination_city, duration, price])
        else:
            # Log failed requests
            print(f"Failed to fetch data for {formatted_date}: {response.status_code}")

        # Include a pause to respect API rate limits
        time.sleep(1)
